# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[R_Cabral_98]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @R_Cabral_98


#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Burguer King'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
dados=pd.read_excel('Burguer King.xlsx')

In [8]:
dados.head()

,Treinamento,Relevante
0,"a frase ""vamos pro burguer king?"" https://t.co...",não
1,matei aquela vontade de comer burguer king,sim
2,rt @feijawn: mermão eu queria mermo era um bur...,sim
3,vou passar a ir ao burguer king no shopping só...,sim
4,o mundo é um burguer king e eu sinto tanta fome,não


In [9]:
#pegando os dados de Treinamento
dados_treinamento = pd.read_excel('Burguer King.xlsx', sheetname="Treinamento")
dados_treinamento = dados_treinamento.loc[:,["Treinamento", "Relevante"]]

#pegando os dados de Teste
dados_teste = pd.read_excel('Burguer King.xlsx', sheetname="Teste")
dados_teste = dados_teste.loc[:,["Teste", "Relevante"]]

In [10]:
#dados_relevante=dados[dados.Relevante=="sim"]
#dados_irrelevante=dados[dados.Relevante=="não"]

In [11]:
#sub_df.iloc[0]['A'] #comando para valor de célula

In [12]:
#captando as frases em uma lista 

def quebrando_tab_relev(dados):
    frases_relevante=[]
    for x in range(len(dados.index)):
        if dados.iloc[x][1]=="sim":
            w=dados.iloc[x][0]
            w=w.upper()
            frases_relevante.append(w)

#print(frases)
    palavras_relevante=[]
    for x in range(len(frases_relevante)):
        w=frases_relevante[x].split(" ")
        palavras_relevante.append(w)
    return palavras_relevante
#print(palavras)


In [13]:
#captando as frases em uma lista 

def quebrando_tab_nr(dados):
    frases_irrelevante=[]
    for x in range(len(dados.index)):
        if dados.iloc[x][1]=="não":
            w=dados.iloc[x][0]
            w=w.upper()
            frases_irrelevante.append(w)

#print(frases)
    palavras_irrelevante=[]
    for x in range(len(frases_irrelevante)):
        w=frases_irrelevante[x].split(" ")
        palavras_irrelevante.append(w)
    return palavras_irrelevante
#print(palavras)


In [14]:
def quebrando_tab_geral(dados):
    frases=[]
    
    for x in range(len(dados.index)):
        w=dados.iloc[x][0]
        w=w.upper()
        frases.append(w)

    palavras=[]
    
    for x in range(len(frases)):
        w=frases[x].split(" ")
        palavras.append(w)
    return palavras

In [15]:
palavras_relev_trei=quebrando_tab_relev(dados_treinamento)    #lista de listas de palavras
palavras_relev_teste=quebrando_tab_relev(dados_teste)    

palavras_nrelev_trei=quebrando_tab_nr(dados_treinamento)    #lista de listas de palavras
palavras_nrelev_teste=quebrando_tab_nr(dados_teste)    


In [16]:
#print(len(palavras))
#print (palavras)

In [17]:
#porcentagens no treino:
porcent_Treino = dados_treinamento.Relevante.value_counts(True)

print("chance de ser relevante:")
porcent_Treino.loc["sim"]

chance de ser relevante:


0.42999999999999999

In [18]:
print("chance de ser irrelevante:")
porcent_Treino.loc["não"]

chance de ser irrelevante:


0.56999999999999995

In [19]:
numbers=[0,1,2,3,4,5,6,7,8,9]
for w in range(len(numbers)):
    x=str(numbers[w])
    numbers[w]=x


In [20]:


import string #recebe lista de palavras e retira pontuação
punct = set(string.punctuation)
conectivos=["DA","DE","DO", "A", "E", "O","DAS", "DOS", "AS", "OS", "AO", "EM", "ME", "NA", "NO", "NAS", "NOS", "TE" "MIM"]


def Remove_pont(palavras):
    lista_p=[]     #transformar lista de listas em lista de palavras
    
    for w in range(len(palavras)):
        for h in (palavras[w]):
            n=0
            if h[0:4]=="HTPPS":       #remove links
                n=n+1
            
            if h in conectivos:    #remove certas preposições e conectivos
                n=n+1
            
            for y in h:
                if y in numbers:       #remove números
                    n=n+1
                if y in punct:       #remove pontuação
                    h=h.replace(y, "")

            if len(h)<2:         #remove caractéres dispensáveis e sinais
                n=n+1
            
            if n==0:    
                lista_p.append(h)
            
    return lista_p

#print(palavras_relev_trei)

In [21]:
def filt(lista): #recebe lista de palavras e filtra
    lista_p=[]     #transformar lista de listas em lista de palavras
    
    for h in (lista):
        n=0
        if h[0:4]=="HTPPS":       #remove links
            n=n+1
        
        if h in conectivos:    #remove certas preposições e conectivos
            n=n+1
            
        for y in h:
            
            if y in numbers:       #remove números
                n=n+1
            
            if y in punct:       #remove pontuação
                h=h.replace(y, "")

        if len(h)<2:         #remove caractéres dispensáveis e sinais
            n=n+1
        
        if n==0:    
            lista_p.append(h)
        
    
    return lista_p

In [22]:
#removendo pontuação
palavras_rel_trei=Remove_pont(palavras_relev_trei)
palavras_rel_teste=Remove_pont(palavras_relev_teste)

palavras_nrel_trei=Remove_pont(palavras_nrelev_trei)
palavras_nrel_teste=Remove_pont(palavras_nrelev_teste)

In [23]:
#transformando lista de palavras em tabela
palavras_r_trei=pd.Series(palavras_rel_trei)
palavras_r_teste=pd.Series(palavras_rel_teste)

palavras_nr_trei=pd.Series(palavras_nrel_trei)
palavras_nr_teste=pd.Series(palavras_nrel_teste)

In [24]:

frases=[]
for x in range(300):
    w=dados.Treinamento[x]
    w=w.upper()
    frases.append(w)

#print(frases)

palavras=[]
for x in range(len(frases)):
    w=frases[x].split(" ")
    palavras.append(w)

#print(palavras)    

palavras=Remove_pont(palavras)   #lista de todas as palavras de treinamento

In [25]:
ndp=(len(palavras))
len(palavras)

2958

In [26]:
palavras=pd.Series(palavras)    #transformando lista de palavras em uma series (banco de dados)
#print(palavras)


In [27]:
# número de palavras totais, relevantes e irrelevantes:
palavras_unic=len(palavras.value_counts())
print("número de palavras únicas:")
print(palavras_unic)

palavras_un_rel=len(palavras_r_trei.value_counts())
print("número de palavras únicas relevantes")
print(palavras_un_rel)

palavras_un_nr=len(palavras_nr_trei.value_counts())
print("número de palavras únicas não relevantes")
print(palavras_un_nr)

palavras_un_qtd=(palavras.value_counts())   #todas as palavras e frêq.
palavras_un_rel_qtd=(palavras_r_trei.value_counts()) #palavras relevantes e frêq.
palavras_un_nrel_qtd=(palavras_nr_trei.value_counts()) #palavras não relevantes e frêq.

#há palavras únicas que aparecem tanto na lista de relevantes quanto de não relevantes

número de palavras únicas:
932
número de palavras únicas relevantes
409
número de palavras únicas não relevantes
681


In [28]:
#verificando aparição de palavras:
rep_palv=[]
for h in palavras_un_rel_qtd.index:
    print("Palavra:", h, "Número de aparições:", palavras_un_rel_qtd[h])
    rep_palv.append([h, palavras_un_rel_qtd[h]])

Palavra: BURGUER Número de aparições: 129
Palavra: KING Número de aparições: 128
Palavra: UM Número de aparições: 56
Palavra: EU Número de aparições: 44
Palavra: RT Número de aparições: 39
Palavra: ERA Número de aparições: 25
Palavra: QUERIA Número de aparições: 25
Palavra: MERMÃO Número de aparições: 21
Palavra: MERMO Número de aparições: 21
Palavra: FEIJAWN Número de aparições: 21
Palavra: QUE Número de aparições: 18
Palavra: VONTADE Número de aparições: 14
Palavra: COMER Número de aparições: 13
Palavra: COM Número de aparições: 11
Palavra: SÓ Número de aparições: 10
Palavra: PRA Número de aparições: 10
Palavra: SE Número de aparições: 9
Palavra: MELHOR Número de aparições: 9
Palavra: IR Número de aparições: 8
Palavra: NÃO Número de aparições: 7
Palavra: HOJE Número de aparições: 7
Palavra: QUERO Número de aparições: 7
Palavra: MINHA Número de aparições: 6
Palavra: MC Número de aparições: 6
Palavra: VOU Número de aparições: 6
Palavra: FUI Número de aparições: 6
Palavra: TÔ Número de 

Palavra: MALUAFD Número de aparições: 1
Palavra: IGUAÇU Número de aparições: 1
Palavra: PAGAR Número de aparições: 1
Palavra: VAMOS Número de aparições: 1
Palavra: SOU Número de aparições: 1
Palavra: DESGASTAR Número de aparições: 1
Palavra: ATÉ Número de aparições: 1
Palavra: PRONTO Número de aparições: 1
Palavra: PRECISAMOS Número de aparições: 1
Palavra: QND Número de aparições: 1
Palavra: OUTROS Número de aparições: 1
Palavra: JÁ Número de aparições: 1
Palavra: TOMASDBOLETA Número de aparições: 1
Palavra: MATAR Número de aparições: 1
Palavra: HORAS Número de aparições: 1
Palavra: NOTA Número de aparições: 1
Palavra: BAITA Número de aparições: 1
Palavra: ESTADOS Número de aparições: 1
Palavra: TIVER Número de aparições: 1
Palavra: ALGO Número de aparições: 1
Palavra: GENTE Número de aparições: 1
Palavra: TENHA Número de aparições: 1
Palavra: DESEJANDO Número de aparições: 1
Palavra: COMPROU Número de aparições: 1
Palavra: GANHAR Número de aparições: 1
Palavra: NADA Número de apariçõ

## Programando classificador:

In [29]:
# Prob_frase=prob_palav1*prob_palav2*prob_palavN    #probabilidade da frase ser relevante

#probabilidade de ser relevante no treino = 43% = 0,43

#probabilidade de ser irrelevante no treino = 57% = 0,57 

#prob_palav =  qtd da palavra em lista relevante / número de palavras relevantes total
#prob_palav_com_laplace_smoothing =  (qtd da palavra em lista relevante+1) / (número de palavras relevantes total + número de palavras únicas)



In [30]:
prob_cd_palavra=[]
for h in palavras_un_qtd.index:
    
    if h in palavras_un_rel_qtd:
        prob_d_ser_relev=((palavras_un_rel_qtd[h])+1)/((len(palavras_rel_trei))+palavras_unic)
    
    if h not in palavras_un_rel_qtd:
        prob_d_ser_relev=(1)/((len(palavras_rel_trei))+palavras_unic)
    
    prob_cd_palavra.append([h, prob_d_ser_relev])

#calculou-se a probabilidade de cada palavra da lista de relevantes, de ser relevante

In [31]:
#transformando em função de probabilidade de ser relativa:

def prob_cd_p_rel(palavras_unicas, palavras_unicas_relativas, n_de_palavras_unicas, lista_de_palavras_relevantes):
    
    prob_cd_palavra=[]
    for h in palavras_unicas.index:
    
        if h in palavras_unicas_relativas:
            prob_d_ser_relev=((palavras_unicas_relativas[h])+1)/((len(lista_de_palavras_relevantes))+n_de_palavras_unicas)
    
        if h not in palavras_unicas:
            prob_d_ser_relev=(1)/((len(lista_de_palavras_relevantes))+n_de_palavras_unicas)
    
        prob_cd_palavra.append ([h, prob_d_ser_relev])
        
    return(prob_cd_palavra)

In [32]:
#print(prob_cd_palavra)
rel_pal=pd.Series(prob_cd_palavra) #lista de probabilidade de cada palavra de ser relevante

In [33]:
prob_nr_pal=[]
for h in palavras_un_qtd.index:
    
    if h in palavras_un_nrel_qtd:
        prob_d_n_ser_relev = (palavras_un_nrel_qtd[h]+1)/((len(palavras_nrel_trei))+palavras_unic)
    
    if h not in palavras_un_nrel_qtd:
        prob_d_n_ser_relev = (1)/((len(palavras_nrel_trei))+palavras_unic)
    
    prob_nr_pal.append([h, prob_d_n_ser_relev])

#calculou-se a probabilidade de cada palavra da lista de não relevantes, de não ser relevante

In [34]:
#transformando em função para calculo de probabilidade de ser não relativa:

def prob_cd_p_n_rel(palavras_unicas, palavras_unicas_n_relativas, n_de_palavras_unicas, lista_de_palavras_n_relevantes):
    
    prob_cd_palavra=[]
    for h in palavras_unicas.index:
    
        if h in palavras_unicas_n_relativas:
            prob_d_ser_relev=((palavras_unicas_n_relativas[h])+1)/((len(lista_de_palavras_n_relevantes))+n_de_palavras_unicas)
    
        if h not in palavras_unicas:
            prob_d_ser_relev=(1)/((len(lista_de_palavras_n_relevantes))+n_de_palavras_unicas)
    
        prob_cd_palavra.append([h, prob_d_ser_relev])
    
    return(prob_cd_palavra)

In [35]:
nrel_pal=pd.Series(prob_nr_pal)  #lista da probabilidade de cada palavra de não ser relevante

In [39]:
#Baseando-se nos códigos e funções anteriores será construida uma que recebe uma tabela (dataframe) e retorna a
#relevância de cada frase.

def relevancia(data_base):
    
    frases=quebrando_tab_geral(data_base) #agora temos uma lista de listas de palavras, sendo cada lista uma frase separada em palavras
    
    for h in range(len(frases)):    #limpando frases
        frases[h]=filt(frases[h])
   
              #######    
    
    
    relevancia_frase=[]
    for w in range(len(frases)):    #passando pelas frases calculando as probabilidades
        pr=[]
        pnr=[]
        rele=1
        nrele=1
        
        for x in frases[w]:
            
            for t in range(len(rel_pal)):
                if x in rel_pal[t]:
                    p=rel_pal[t][1]
                    pr.append(p)
            
            for t in range(len(nrel_pal)):
                if x in nrel_pal[t]:
                    p=nrel_pal[t][1]
                    pnr.append(p)
                    
        for k in range(len(pr)):
            rele=rele*pr[k]
        
        for k in range(len(pnr)):
            nrele=nrele*pnr[k]
    
        
        pdsr=rele
        pdnsr=nrele
        
        if pdsr>pdnsr:
            relevancia_frase.append("sim")
        
        if pdsr<pdnsr:
            relevancia_frase.append("não")
            
    return relevancia_frase

# A função pode ser otimizada ainda.

In [40]:
relevancia_dados_treino=relevancia(dados_treinamento)

In [42]:
#print(relevancia_dados_treino)

In [45]:
dados_treinamento["Relevante_programa"]=relevancia_dados_treino
dados_treinamento
#comparando relevancia do programa com a relevancia manual.

,Treinamento,Relevante,Relevante_programa
0,"a frase ""vamos pro burguer king?"" https://t.co...",não,não
1,matei aquela vontade de comer burguer king,sim,sim
2,rt @feijawn: mermão eu queria mermo era um bur...,sim,sim
3,vou passar a ir ao burguer king no shopping só...,sim,sim
4,o mundo é um burguer king e eu sinto tanta fome,não,sim
5,e minha mãe q veio aqui no quarto perguntar se...,não,sim
6,café da manhã com desconto no burguer king - e...,sim,sim
7,rt @feijawn: mermão eu queria mermo era um bur...,sim,sim
8,rt @amaandamooura_: afasta de mim essa vontade...,não,não
9,@maluafd pra mim é dia de burguer king,sim,sim


In [50]:
#comparando a tabela de relevancia do programa com a manual:

positivos_vdd=0
positivos_fal=0
negativos_vdd=0
negativos_fal=0

for h in range(len(dados_treinamento)):
    
    prog=dados_treinamento.iloc[h][2]
    manual=dados_treinamento.iloc[h][1]
    
    if prog=="sim":
        if prog==manual:
            positivos_vdd=positivos_vdd+1
        
        if prog!=manual:
            positivos_fal=positivos_fal+1
    
    if prog=="não":
        if prog==manual:
            negativos_vdd=negativos_vdd+1
        
        if prog!=manual:
            negativos_fal=negativos_fal+1
    

In [58]:
#análise:

print("número de positivos verdadeiros:")
print(positivos_vdd)
print("")

print("número de positivos falsos:")
print(positivos_fal)
print("")

print("número de negativos verdadeiros:")
print(negativos_vdd)
print("")

print("número de negativos falsos:")
print(negativos_fal)
print("")

print("porcentagem de acerto:")
acerto=(positivos_vdd+negativos_vdd)/300
acertop=acerto*100
print(acertop,"%")

número de positivos verdadeiros:
125

número de positivos falsos:
17

número de negativos verdadeiros:
154

número de negativos falsos:
4

porcentagem de acerto:
93.0 %


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [72]:
#Fazendo algumas alterações para a função ser mais universal

def relevancia(data_base, dados_rel, dados_n_rel):
    
    frases=quebrando_tab_geral(data_base) #agora temos uma lista de listas de palavras, sendo cada lista uma frase separada em palavras
    
    for h in range(len(frases)):    #limpando frases
        frases[h]=filt(frases[h])
   
              #######    
    
    
    relevancia_frase=[]
    for w in range(len(frases)):    #passando pelas frases calculando as probabilidades
        pr=[]
        pnr=[]
        rele=1
        nrele=1
        
        for x in frases[w]:
            
            for t in range(len(dados_rel)):
                if x in dados_rel[t]:
                    p=dados_rel[t][1]
                    pr.append(p)
            
            for t in range(len(dados_n_rel)):
                if x in dados_n_rel[t]:
                    p=dados_n_rel[t][1]
                    pnr.append(p)
                    
        for k in range(len(pr)):
            rele=rele*pr[k]
        
        for k in range(len(pnr)):
            nrele=nrele*pnr[k]
    
        
        pdsr=rele
        pdnsr=nrele
        
        if pdsr>pdnsr:
            relevancia_frase.append("sim")
        
        if pdsr<pdnsr:
            relevancia_frase.append("não")
            
    return relevancia_frase

# A função pode ser otimizada ainda.

In [73]:
frases=[]
for x in range(200):
    w=dados_teste.Teste[x]
    w=w.upper()
    frases.append(w)

#print(frases)

palavras=[]
for x in range(len(frases)):
    w=frases[x].split(" ")
    palavras.append(w)

#print(palavras)    

palavras=Remove_pont(palavras)#lista de todas as palavras de treinamento
palavras=pd.Series(palavras)

In [74]:
# número de palavras totais, relevantes e irrelevantes:
palavras_unic=len(palavras.value_counts())
print("número de palavras únicas:")
print(palavras_unic)

palavras_un_rel=len(palavras_r_teste.value_counts())
print("número de palavras únicas relevantes")
print(palavras_un_rel)

palavras_un_nr=len(palavras_nr_teste.value_counts())
print("número de palavras únicas não relevantes")
print(palavras_un_nr)

palavras_un_qtd=(palavras.value_counts())   #todas as palavras e frêq.
palavras_un_rel_qtd=(palavras_r_trei.value_counts()) #palavras relevantes e frêq.
palavras_un_nrel_qtd=(palavras_nr_trei.value_counts()) #palavras não relevantes e frêq.

#há palavras únicas que aparecem tanto na lista de relevantes quanto de não relevantes

número de palavras únicas:
720
número de palavras únicas relevantes
229
número de palavras únicas não relevantes
587


In [75]:
rel_pal=prob_cd_p_rel(palavras_un_qtd, palavras_un_rel_qtd, palavras_unic, palavras_rel_teste)

nrel_pal=prob_cd_p_rel(palavras_un_qtd, palavras_un_nrel_qtd, palavras_unic, palavras_nr_teste)

In [76]:
#calculando a relevancia das frases da lista de teste:
relevancia_dados_teste = relevancia(dados_teste, rel_pal, nrel_pal)

In [77]:
dados_teste["Relevante_programa"]=relevancia_dados_teste
dados_teste
#comparando relevancia do programa com a relevancia manual.

,Teste,Relevante,Relevante_programa
0,não sei como vcs gostam desses hambúrgueres do...,não,não
1,@lucas_lira vai no mac donald's e pergunta se ...,não,sim
2,rt @gomessss10: mc donald's dá 1000-0 ao burgu...,não,não
3,gostei de um vídeo @youtube https://t.co/sgkjo...,não,não
4,dica pra comer no burguer king brasil\n\npeça ...,sim,sim
5,ai que fome de lanches do burguer king,sim,não
6,rt @bfrondon_: burguer king melhor que mc dona...,sim,sim
7,"rt @andresamell15: estou com fome, fome de bur...",sim,sim
8,maria: comi 2 hambúrguer do burguer king\n mar...,não,sim
9,rt @feijawn: mermão eu queria mermo era um bur...,sim,sim


In [78]:
#comparando a tabela de relevancia do programa com a manual:

positivos_vdd=0
positivos_fal=0
negativos_vdd=0
negativos_fal=0

for h in range(len(dados_teste)):
    
    prog=dados_teste.iloc[h][2]
    manual=dados_teste.iloc[h][1]
    
    if prog=="sim":
        if prog==manual:
            positivos_vdd=positivos_vdd+1
        
        if prog!=manual:
            positivos_fal=positivos_fal+1
    
    if prog=="não":
        if prog==manual:
            negativos_vdd=negativos_vdd+1
        
        if prog!=manual:
            negativos_fal=negativos_fal+1
    

In [81]:
#análise:

print("número de positivos verdadeiros:")
print(positivos_vdd)
print("")

print("número de positivos falsos:")
print(positivos_fal)
print("")

print("número de negativos verdadeiros:")
print(negativos_vdd)
print("")

print("número de negativos falsos:")
print(negativos_fal)
print("")

print("porcentagem de acerto:")
acerto=(positivos_vdd+negativos_vdd)/200
acertop=acerto*100
print(acertop,"%")

número de positivos verdadeiros:
71

número de positivos falsos:
103

número de negativos verdadeiros:
19

número de negativos falsos:
7

porcentagem de acerto:
45.0 %


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
